In [1]:
# ==================== COMPLETE ENHANCED MEME ANALYSIS PIPELINE ====================
# FIXED VERSION: Prints "Task A Weighted F1" in Validation Logs & Final Summary
# ==================================================================================

# ==================== PART 0: SETUP & DEPENDENCIES ====================
import sys, subprocess, os, json, zipfile, shutil, random, warnings
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler, WeightedRandomSampler
from torch.cuda.amp import GradScaler, autocast
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm
import yaml
import pickle 

warnings.filterwarnings('ignore')

print("=" * 80)
print("INSTALLING DEPENDENCIES...")
print("=" * 80)

# Install required packages
subprocess.check_call([
    sys.executable, "-m", "pip", "install", "-q",
    "transformers>=4.40.0", "accelerate", "torch", "timm",
    "scikit-learn", "pandas", "matplotlib", "seaborn",
    "huggingface_hub>=0.18.0", "gdown", "iterative-stratification"
])

# Import additional packages
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    mean_squared_error, mean_absolute_error, f1_score, fbeta_score
)
from transformers import AutoModel, AutoTokenizer, CLIPModel
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

print("✓ All dependencies installed\n")


# ==================== PART 1: DATA PREPARATION ====================
print("=" * 80)
print("PART 1: DATA PREPARATION")
print("=" * 80)

# Download and extract dataset
print("\nDownloading dataset...")
subprocess.run(["gdown", "1jEJ2nf5CDJknq80ogzU-Uyz7jbBi-1LZ", "--fuzzy"],
               check=False, capture_output=True)

print("Extracting dataset...")
zip_files = [f for f in os.listdir('.') if f.endswith('.zip')]
if zip_files:
    subprocess.run(["unzip", "-q", "-o", zip_files[0]], check=False, capture_output=True)

# Download additional files
subprocess.run([
    "gdown", "--folder", "19yaav8ORSVj9DeJUaHKq1H3HtVnkClBw", "--remaining-ok"
], check=False, capture_output=True)

# Extract password-protected archive
print("Extracting protected archive...")
zip_path = '/kaggle/working/Memotion 3/memotion3.zip'
extract_to = '/kaggle/working/'
password = b'memotion3taskaaai@22'

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to, pwd=password)
    print(f"✓ Extracted to: {extract_to}")

# Setup paths
ORIGINAL_TRAIN_IMG_DIR = '/kaggle/working/trainImages/'
ORIGINAL_CSV_PATH = '/kaggle/working/memotion3/train.csv'
VALIDATION_SPLIT_RATIO = 0.15

OUTPUT_BASE_DIR = '/kaggle/working/'
NEW_VAL_DIR = os.path.join(OUTPUT_BASE_DIR, 'validation_images/')
NEW_TRAIN_DIR = os.path.join(OUTPUT_BASE_DIR, 'new_train_images/')

os.makedirs(NEW_VAL_DIR, exist_ok=True)
os.makedirs(NEW_TRAIN_DIR, exist_ok=True)

# Load and process CSV
print("\nLoading CSV file...")
df = pd.read_csv(ORIGINAL_CSV_PATH)

# Detect image column
possible_image_cols = ['image_name', 'image', 'img_name', 'filename', 'Unnamed: 0']
IMAGE_FILENAME_COLUMN = next((col for col in possible_image_cols if col in df.columns), df.columns[0])
print(f"✓ Image column: {IMAGE_FILENAME_COLUMN}")

# Normalize labels
for col in ['offensive', 'motivational', 'humour', 'humor', 'sarcastic', 'sarcasm', 'overall', 'sentiment']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.lower().str.strip()

# Create binary labels
def create_binary_label(value, positive_values):
    if pd.isna(value) or value in ['nan', 'none', '']:
        return 0
    return 1 if value in positive_values else 0

if 'offensive' in df.columns:
    df['offensive_bin'] = df['offensive'].apply(
        lambda x: create_binary_label(x, ['slight', 'very_offensive', 'hateful_offensive'])
    )
else:
    df['offensive_bin'] = 0

if 'motivational' in df.columns:
    df['motivational_bin'] = df['motivational'].apply(
        lambda x: create_binary_label(x, ['motivational'])
    )
else:
    df['motivational_bin'] = 0

if 'humour' in df.columns or 'humor' in df.columns:
    humor_col = 'humour' if 'humour' in df.columns else 'humor'
    df['humor_bin'] = df[humor_col].apply(
        lambda x: create_binary_label(x, ['funny', 'very_funny', 'hilarious'])
    )
else:
    df['humor_bin'] = 0

if 'sarcastic' in df.columns or 'sarcasm' in df.columns:
    sarcasm_col = 'sarcastic' if 'sarcastic' in df.columns else 'sarcasm'
    df['sarcasm_bin'] = df[sarcasm_col].apply(
        lambda x: create_binary_label(x, ['general', 'twisted_meaning', 'very_twisted'])
    )
else:
    df['sarcasm_bin'] = 0

# Stratified split
stratify_columns = ['offensive_bin', 'motivational_bin', 'humor_bin', 'sarcasm_bin']
y_stratify = df[stratify_columns].values

msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=VALIDATION_SPLIT_RATIO, random_state=42)
train_idx, val_idx = next(msss.split(df, y_stratify))

train_df = df.iloc[train_idx].reset_index(drop=True)
val_df = df.iloc[val_idx].reset_index(drop=True)

print(f"\n✓ Stratified split complete:")
print(f"  Training: {len(train_df)} samples")
print(f"  Validation: {len(val_df)} samples")

# Calculate label priors
label_priors = {
    'offensive_pos_rate': float(train_df['offensive_bin'].sum() / len(train_df)),
    'motivational_pos_rate': float(train_df['motivational_bin'].sum() / len(train_df)),
    'humor_pos_rate': float(train_df['humor_bin'].sum() / len(train_df)),
    'sarcasm_pos_rate': float(train_df['sarcasm_bin'].sum() / len(train_df))
}

priors_path = os.path.join(OUTPUT_BASE_DIR, 'label_priors.json')
with open(priors_path, 'w') as f:
    json.dump(label_priors, f, indent=2)

print(f"\n✓ Label priors calculated:")
for key, val in label_priors.items():
    print(f"  {key}: {val:.4f}")

# Copy images
def copy_images(df_subset, dest_dir, source_dir, image_col):
    copied = 0
    missing = 0
    
    for idx in tqdm(df_subset[image_col], desc=f"Copying to {dest_dir}"):
        filename = str(idx)
        if not any(filename.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.gif']):
            for ext in ['.jpg', '.jpeg', '.png']:
                test_path = os.path.join(source_dir, f"{filename}{ext}")
                if os.path.exists(test_path):
                    filename = f"{filename}{ext}"
                    break
            else:
                filename = f"{filename}.jpg"
        
        source_path = os.path.join(source_dir, filename)
        if os.path.exists(source_path):
            shutil.copy(source_path, os.path.join(dest_dir, filename))
            copied += 1
        else:
            missing += 1
    
    return copied, missing

print("\nCopying images...")
copied_val, missing_val = copy_images(val_df, NEW_VAL_DIR, ORIGINAL_TRAIN_IMG_DIR, IMAGE_FILENAME_COLUMN)
copied_train, missing_train = copy_images(train_df, NEW_TRAIN_DIR, ORIGINAL_TRAIN_IMG_DIR, IMAGE_FILENAME_COLUMN)

print(f"✓ Validation: {copied_val} copied, {missing_val} missing")
print(f"✓ Training: {copied_train} copied, {missing_train} missing")

# Save CSVs
train_csv_path = os.path.join(OUTPUT_BASE_DIR, 'train_split.csv')
val_csv_path = os.path.join(OUTPUT_BASE_DIR, 'validation_split.csv')

train_df.to_csv(train_csv_path, index=False)
val_df.to_csv(val_csv_path, index=False)

print(f"\n✓ Saved train CSV: {train_csv_path}")
print(f"✓ Saved validation CSV: {val_csv_path}")
print("\n✅ DATA PREPARATION COMPLETE\n")


# ==================== PART 2: CONFIGURATION ====================
print("=" * 80)
print("PART 2: CONFIGURATION (BALANCED STRATEGY)")
print("=" * 80)

CONFIG_YAML = """
TEXT_MODEL: "google/muril-base-cased"
IMAGE_MODEL: "openai/clip-vit-base-patch32"
TEXT_DIM: 768
IMAGE_DIM: 768
FUSION_DIM: 512
FUSION_OUT_DIM: 512

MAX_LEN: 128
IMG_SIZE: 224
BATCH_SIZE: 16
GRADIENT_ACCUMULATION_STEPS: 2

# --- UPDATED LEARNING RATES ---
LR_HEADS: 0.0003
LR_BACKBONE: 0.00001
WEIGHT_DECAY: 0.01

EPOCHS: 20
FINE_TUNE_EPOCHS: 5
SEED: 42
DEVICE: "cuda"
CHECKPOINT_PATH: "/kaggle/working/checkpoints"

NUM_SENTIMENT_CLASSES: 5
NUM_EMOTION_CLASSES: 4

USE_ORDINAL_REGRESSION: true
ORDINAL_LINK: "logit"

# --- UPDATED LOSS WEIGHTS ---
LOSS_WEIGHTS:
  sentiment: 1.0
  emotion: 4.0
  intensity: 0.1

# --- UPDATED ASL SETTINGS ---
ASL_GAMMA_NEG: 4.0
ASL_GAMMA_POS: 0.25
ASL_CLIP: 0.04
ASL_PRIOR_TAU: 1.2

EMOTION_LABELS: ["humor", "sarcasm", "offensive", "motivational"]

# --- THRESHOLDS & TUNING ---
EMO_THRESHOLDS: [0.5, 0.5, 0.25, 0.25]
TUNE_EMOTION_THRESHOLDS: true

POOLING: "mean"
USE_AMP: false
GRADIENT_CLIP: 1.0
SCHEDULER: "cosine"
UNFREEZE_BACKBONE_EPOCH: 2
UNFREEZE_LAYERS: 3

# --- SAMPLING STRATEGY ---
MOTIVATIONAL_OVERSAMPLE_FACTOR: 10.0
MOTIVATIONAL_BATCH_RATIO: 0.30

CROSS_ATTN_HEADS: 8
CROSS_ATTN_DROPOUT: 0.1

SENTIMENT_MAP_REV:
  0: "very_positive"
  1: "positive"
  2: "neutral"
  3: "negative"
  4: "very_negative"
"""

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

cfg = yaml.safe_load(CONFIG_YAML)
set_seed(cfg['SEED'])

# Load priors
priors_path = '/kaggle/working/label_priors.json'
if os.path.exists(priors_path):
    with open(priors_path, 'r') as f:
        priors = json.load(f)

    cfg['EMO_PRIORS'] = [
        priors['humor_pos_rate'],
        priors['sarcasm_pos_rate'],
        priors['offensive_pos_rate'],
        priors['motivational_pos_rate']
    ]
else:
    print("Warning: Label priors not found. Using defaults.")
    cfg['EMO_PRIORS'] = [0.1, 0.1, 0.1, 0.1]

device = torch.device(cfg['DEVICE'] if torch.cuda.is_available() else 'cpu')

print(f"\n✓ Configuration loaded:")
print(f"  Device: {device}")
print(f"  Motivational Batch Ratio: {cfg['MOTIVATIONAL_BATCH_RATIO']}")
print(f"  Fine-tune Epochs: {cfg['FINE_TUNE_EPOCHS']}")


# ==================== PART 3: MODEL COMPONENTS ====================
print("=" * 80)
print("MODEL COMPONENTS")
print("=" * 80)

class OrdinalRegressionHead(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.num_thresholds = num_classes - 1
        
        self.projection = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 1)
        )
        
        initial_thresholds = torch.linspace(-2, 2, self.num_thresholds)
        self.thresholds = nn.Parameter(initial_thresholds)
    
    def forward(self, x):
        score = self.projection(x).squeeze(-1)
        ordered_thresholds = torch.cumsum(F.softplus(self.thresholds), dim=0)
        cumulative_logits = ordered_thresholds.unsqueeze(0) - score.unsqueeze(1)
        cumulative_probs = torch.sigmoid(cumulative_logits)
        
        batch_size = cumulative_probs.size(0)
        class_probs = torch.zeros(batch_size, self.num_classes, device=x.device)
        
        class_probs[:, 0] = cumulative_probs[:, 0]
        for k in range(1, self.num_thresholds):
            class_probs[:, k] = cumulative_probs[:, k] - cumulative_probs[:, k-1]
        class_probs[:, -1] = 1.0 - cumulative_probs[:, -1]
        class_probs = torch.clamp(class_probs, min=1e-7, max=1.0)
        
        return {'cumulative_logits': cumulative_logits, 'class_probs': class_probs}

class HybridSentimentHead(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.ordinal_head = OrdinalRegressionHead(input_dim, num_classes)
        self.ce_head = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        ordinal_outputs = self.ordinal_head(x)
        ce_logits = self.ce_head(x)
        return {
            'cumulative_logits': ordinal_outputs['cumulative_logits'],
            'class_probs': ordinal_outputs['class_probs'],
            'ce_logits': ce_logits
        }

class CrossAttentionFusion(nn.Module):
    def __init__(self, dim, num_heads=8, dropout=0.1):
        super().__init__()
        self.text_to_image_attn = nn.MultiheadAttention(dim, num_heads, dropout, batch_first=True)
        self.image_to_text_attn = nn.MultiheadAttention(dim, num_heads, dropout, batch_first=True)
        self.text_norm = nn.LayerNorm(dim)
        self.image_norm = nn.LayerNorm(dim)
        self.text_ffn = nn.Sequential(
            nn.Linear(dim, dim * 4), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(dim * 4, dim), nn.Dropout(dropout)
        )
        self.image_ffn = nn.Sequential(
            nn.Linear(dim, dim * 4), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(dim * 4, dim), nn.Dropout(dropout)
        )
        self.ffn_norm_text = nn.LayerNorm(dim)
        self.ffn_norm_image = nn.LayerNorm(dim)
    
    def forward(self, text_emb, image_emb):
        text_seq = text_emb.unsqueeze(1)
        image_seq = image_emb.unsqueeze(1)
        
        text_attended, _ = self.text_to_image_attn(text_seq, image_seq, image_seq)
        text_out = self.text_norm(text_emb + text_attended.squeeze(1))
        
        image_attended, _ = self.image_to_text_attn(image_seq, text_seq, text_seq)
        image_out = self.image_norm(image_emb + image_attended.squeeze(1))
        
        text_final = self.ffn_norm_text(text_out + self.text_ffn(text_out))
        image_final = self.ffn_norm_image(image_out + self.image_ffn(image_out))
        
        return text_final, image_final

class EnhancedFusionModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        self.text_model = AutoModel.from_pretrained(cfg['TEXT_MODEL'])
        clip_model = CLIPModel.from_pretrained(cfg['IMAGE_MODEL'])
        self.image_model = clip_model.vision_model
        
        self._freeze_encoders()
        
        self.text_proj = nn.Linear(cfg['TEXT_DIM'], cfg['FUSION_DIM'])
        self.image_proj = nn.Linear(cfg['IMAGE_DIM'], cfg['FUSION_DIM'])
        
        self.cross_attention = CrossAttentionFusion(
            dim=cfg['FUSION_DIM'],
            num_heads=cfg['CROSS_ATTN_HEADS'],
            dropout=cfg['CROSS_ATTN_DROPOUT']
        )
        
        fusion_input_dim = cfg['FUSION_DIM'] * 2
        self.fusion_norm = nn.LayerNorm(fusion_input_dim)
        self.fusion_mlp = nn.Sequential(
            nn.Linear(fusion_input_dim, 512), nn.GELU(), nn.Dropout(0.2),
            nn.Linear(512, cfg['FUSION_OUT_DIM']), nn.LayerNorm(cfg['FUSION_OUT_DIM'])
        )
        
        if cfg.get('USE_HYBRID_SENTIMENT_LOSS', False):
            self.sentiment_head = HybridSentimentHead(cfg['FUSION_OUT_DIM'], cfg['NUM_SENTIMENT_CLASSES'])
        else:
            self.sentiment_head = OrdinalRegressionHead(cfg['FUSION_OUT_DIM'], cfg['NUM_SENTIMENT_CLASSES'])
        
        self.emotion_head = nn.Sequential(
            nn.Linear(cfg['FUSION_OUT_DIM'], 256), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(256, cfg['NUM_EMOTION_CLASSES'])
        )
        self.intensity_head = nn.Sequential(
            nn.Linear(cfg['FUSION_OUT_DIM'], 128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 1)
        )
    
    def _freeze_encoders(self):
        for param in self.text_model.parameters():
            param.requires_grad = False
        for param in self.image_model.parameters():
            param.requires_grad = False
    
    def unfreeze_backbone(self, layers_to_unfreeze=3):
        if hasattr(self.text_model, 'encoder') and hasattr(self.text_model.encoder, 'layer'):
            for layer in list(self.text_model.encoder.layer[-layers_to_unfreeze:]):
                for param in layer.parameters():
                    param.requires_grad = True
        
        if hasattr(self.image_model, 'encoder') and hasattr(self.image_model.encoder, 'layers'):
            for layer in list(self.image_model.encoder.layers[-layers_to_unfreeze:]):
                for param in layer.parameters():
                    param.requires_grad = True
    
    def pool_text(self, model_output, attention_mask):
        last_hidden = model_output.last_hidden_state
        if self.cfg['POOLING'] == 'cls':
            return last_hidden[:, 0]
        mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
        sum_embeddings = torch.sum(last_hidden * mask_expanded, 1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask
    
    def forward(self, input_ids, attention_mask, image):
        text_output = self.text_model(input_ids=input_ids, attention_mask=attention_mask)
        text_emb = self.pool_text(text_output, attention_mask)
        
        image_output = self.image_model(pixel_values=image)
        image_emb = image_output.pooler_output
        
        text_proj = self.text_proj(text_emb)
        image_proj = self.image_proj(image_emb)
        
        text_cross, image_cross = self.cross_attention(text_proj, image_proj)
        
        fused = torch.cat([text_cross, image_cross], dim=1)
        fused = self.fusion_norm(fused)
        fused = self.fusion_mlp(fused)
        
        sentiment_outputs = self.sentiment_head(fused)
        emotion_logits = self.emotion_head(fused)
        intensity = self.intensity_head(fused).squeeze(-1)
        
        return {
            'sentiment': sentiment_outputs,
            'emotion_logits': emotion_logits,
            'intensity': intensity
        }

print("✓ Model components defined")


# ==================== PART 4: DATASET ====================
print("=" * 80)
print("DATASET")
print("=" * 80)

class MemeDataset(Dataset):
    def __init__(self, df, tokenizer, image_transform, image_dir, cfg):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.image_transform = image_transform
        self.image_dir = image_dir
        self.cfg = cfg
        self._detect_columns()
    
    def _detect_columns(self):
        cols = self.df.columns.tolist()
        self.image_col = next((c for c in ['image_name', 'image', 'img_name', 'filename', 'Unnamed: 0'] if c in cols), cols[0])
        self.text_col = next((c for c in ['text', 'ocr_text', 'caption', 'OCR_extracted_text'] if c in cols), None)
        self.sentiment_col = next((c for c in ['sentiment', 'overall_sentiment', 'overall'] if c in cols), None)
        
        self.sentiment_map = {'very_positive': 0, 'positive': 1, 'neutral': 2, 'negative': 3, 'very_negative': 4}
        self.humor_map = {'not_funny': 0, 'funny': 1, 'very_funny': 1, 'hilarious': 1}
        self.sarcasm_map = {'not_sarcastic': 0, 'general': 1, 'twisted_meaning': 1, 'very_twisted': 1}
        self.offensive_map = {'not_offensive': 0, 'slight': 1, 'very_offensive': 1, 'hateful_offensive': 1}
        self.motivational_map = {'not_motivational': 0, 'motivational': 1}
    
    def _map_label(self, value, mapping, default=0):
        if pd.isna(value):
            return default
        if isinstance(value, str):
            return mapping.get(value.lower().strip(), default)
        return int(value)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        image_name = str(row[self.image_col])
        if not any(image_name.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
            image_name = f"{image_name}.jpg"
        image_path = os.path.join(self.image_dir, image_name)
        
        try:
            image = Image.open(image_path).convert('RGB')
            image = self.image_transform(image)
        except:
            image = torch.zeros(3, self.cfg['IMG_SIZE'], self.cfg['IMG_SIZE'])
        
        text = str(row.get(self.text_col, '')) if self.text_col else 'No text'
        encoding = self.tokenizer(text, max_length=self.cfg['MAX_LEN'], padding='max_length', truncation=True, return_tensors='pt')
        
        sentiment_val = row.get(self.sentiment_col, 'neutral') if self.sentiment_col else 'neutral'
        sentiment_label = self._map_label(sentiment_val, self.sentiment_map, default=2)
        
        emotion_labels = torch.tensor([
            float(self._map_label(row.get('humour', row.get('humor', 0)), self.humor_map, 0)),
            float(self._map_label(row.get('sarcastic', row.get('sarcasm', 0)), self.sarcasm_map, 0)),
            float(self._map_label(row.get('offensive', 0), self.offensive_map, 0)),
            float(self._map_label(row.get('motivational', 0), self.motivational_map, 0))
        ], dtype=torch.float)
        
        is_extreme = sentiment_label in [0, 4]
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'image': image,
            'sentiment_label': torch.tensor(sentiment_label, dtype=torch.long),
            'emotion_labels': emotion_labels,
            'intensity': torch.tensor(0.5, dtype=torch.float),
            'motivational_flag': emotion_labels[3],
            'extreme_sentiment_flag': torch.tensor(float(is_extreme))
        }

print("✓ Dataset class defined")


# ==================== PART 5: LOSS FUNCTIONS & METRICS ====================
print("\n" + "=" * 80)
print("LOSS FUNCTIONS & METRICS (F2 OPTIMIZATION)")
print("=" * 80)

class EnhancedAsymmetricLoss(nn.Module):
    def __init__(self, gamma_neg=4.0, gamma_pos=0.25, clip=0.04, priors=None, prior_tau=1.2, eps=1e-8):
        super().__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.eps = eps
        self.priors = priors
        self.prior_tau = prior_tau

    def forward(self, logits, targets):
        if self.priors is not None:
            priors_tensor = torch.tensor(self.priors, device=logits.device, dtype=logits.dtype)
            adjustment = self.prior_tau * torch.log(priors_tensor.clamp(min=self.eps))
            logits = logits - adjustment

        xs_pos = torch.sigmoid(logits)
        xs_neg = 1 - xs_pos

        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)

        los_pos = targets * torch.log(xs_pos.clamp(min=self.eps))
        los_neg = (1 - targets) * torch.log(xs_neg.clamp(min=self.eps))

        if self.gamma_neg > 0 or self.gamma_pos > 0:
            pt0 = xs_pos * targets
            pt1 = xs_neg * (1 - targets)
            pt = pt0 + pt1
            one_sided_gamma = self.gamma_pos * targets + self.gamma_neg * (1 - targets)
            one_sided_w = torch.pow(1 - pt, one_sided_gamma)
            loss = -one_sided_w * (los_pos + los_neg)
        else:
            loss = -(los_pos + los_neg)

        # --- UPDATED: MANUAL WEIGHTING FOR MOTIVATIONAL ---
        if logits.size(1) == 4:
            # [Humor, Sarcasm, Offensive, Motivational]
            # Boosting Motivational by 1.7x
            manual_weights = torch.tensor([1.0, 1.0, 1.0, 1.7], device=logits.device, dtype=logits.dtype)
            loss = loss * manual_weights

        return loss.mean()

def ordinal_regression_loss(cumulative_logits, labels):
    batch_size = labels.size(0)
    num_thresholds = cumulative_logits.size(1)
    target_cumulative = torch.zeros_like(cumulative_logits)
    for i in range(batch_size):
        y = int(labels[i].item())
        if y < num_thresholds:
            target_cumulative[i, y:] = 1.0
    return F.binary_cross_entropy_with_logits(cumulative_logits, target_cumulative, reduction='mean')

def combined_loss(outputs, batch, cfg, emotion_loss_fn):
    loss_sent = ordinal_regression_loss(outputs['sentiment']['cumulative_logits'], batch['sentiment_label'])
    loss_emotion = emotion_loss_fn(outputs['emotion_logits'], batch['emotion_labels'])
    loss_intensity = F.smooth_l1_loss(outputs['intensity'], batch['intensity'])
    
    total_loss = (
        cfg['LOSS_WEIGHTS']['sentiment'] * loss_sent +
        cfg['LOSS_WEIGHTS']['emotion'] * loss_emotion +
        cfg['LOSS_WEIGHTS']['intensity'] * loss_intensity
    )
    return total_loss, loss_sent, loss_emotion, loss_intensity

def tune_emotion_thresholds(emotion_logits, emotion_labels, num_thresholds=100):
    emotion_probs = torch.sigmoid(emotion_logits).cpu().numpy()
    emotion_true = emotion_labels.cpu().numpy()
    
    num_emotions = emotion_probs.shape[1]
    optimal_thresholds = []
    
    for emo_idx in range(num_emotions):
        best_score = 0.0
        best_threshold = 0.5
        
        y_true = emotion_true[:, emo_idx]
        y_prob = emotion_probs[:, emo_idx]
        
        # --- UPDATED: Targeted F2 Optimization for Motivational (Idx 3) ---
        if emo_idx == 3:
            # Specific Band: 0.15 to 0.60
            search_range = np.linspace(0.15, 0.60, num_thresholds)
        else:
            search_range = np.linspace(0.1, 0.9, num_thresholds)
        
        for threshold in search_range:
            y_pred = (y_prob >= threshold).astype(float)
            
            if emo_idx == 3:
                # OPTIMIZE F2 (Recall-Weighted) for Motivational
                score = fbeta_score(y_true, y_pred, beta=2.0, zero_division=0)
            else:
                score = f1_score(y_true, y_pred, zero_division=0)
            
            if score > best_score:
                best_score = score
                best_threshold = threshold
        
        optimal_thresholds.append(best_threshold)
        print(f"  Emotion {emo_idx}: threshold={best_threshold:.3f}, Score={best_score:.4f}")
    
    return optimal_thresholds

def compute_metrics(sentiment_outputs, sentiment_labels, emotion_logits, emotion_labels, thresholds):
    # Sentiment
    class_probs = sentiment_outputs['class_probs']
    y_pred = torch.argmax(class_probs, dim=1).cpu().numpy()
    y_true = sentiment_labels.cpu().numpy()
    sent_acc = accuracy_score(y_true, y_pred)
    _, _, sent_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro', zero_division=0)
    sent_mae = mean_absolute_error(y_true, y_pred)
    sent_1off = np.mean(np.abs(y_true - y_pred) <= 1)

    # NEW: Task A Style Metrics (Collapse 5 classes -> 3 classes)
    task_a_map = {0: 0, 1: 0, 2: 1, 3: 2, 4: 2}
    y_true_3class = np.array([task_a_map[val] for val in y_true])
    y_pred_3class = np.array([task_a_map[val] for val in y_pred])
    sent_f1_taskA_weighted = f1_score(y_true_3class, y_pred_3class, average='weighted', zero_division=0)

    # Emotions
    emo_probs = torch.sigmoid(emotion_logits).cpu().numpy()
    emo_true = emotion_labels.cpu().numpy()
    thresholds = np.array(thresholds)
    emo_pred = (emo_probs >= thresholds).astype(float)
    
    _, _, emo_f1, _ = precision_recall_fscore_support(emo_true, emo_pred, average='samples', zero_division=0)
    
    per_emotion_f1 = []
    for i in range(emo_true.shape[1]):
        f1 = f1_score(emo_true[:, i], emo_pred[:, i], zero_division=0)
        per_emotion_f1.append(f1)

    return {
        'sentiment_accuracy': sent_acc,
        'sentiment_f1': sent_f1,
        'sentiment_taskA_weighted_f1': sent_f1_taskA_weighted,
        'sentiment_mae': sent_mae,
        'sentiment_1off_accuracy': sent_1off,
        'emotion_f1': emo_f1,
        'per_emotion_f1': per_emotion_f1
    }

print("✓ Loss functions and metrics defined")


# ==================== PART 6: TRAINER (UPDATED) ====================
print("=" * 80)
print("TRAINER (TWO-STAGE TRAINING + BALANCED SAMPLER + FINAL SUMMARY)")
print("=" * 80)

class Trainer:
    def __init__(self, model, cfg, train_loader, val_loader, device, emotion_loss_fn):
        self.model = model
        self.cfg = cfg
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.emotion_loss_fn = emotion_loss_fn

        self.optimizer = self.make_optimizer()
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=cfg['EPOCHS'])
        self.scaler = GradScaler() if cfg['USE_AMP'] else None
        self.best_metric = -float('inf')
        self.optimal_thresholds = cfg['EMO_THRESHOLDS'].copy()
        
        # Tracking best epoch details
        self.best_epoch_metrics = None
        self.best_epoch_num = -1

        # Early Stopping
        self.patience = 3
        self.counter = 0

    def make_optimizer(self):
        head_params = []
        backbone_params = []
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                if 'text_model' in name or 'image_model' in name:
                    backbone_params.append(param)
                else:
                    head_params.append(param)
        param_groups = [{'params': head_params, 'lr': self.cfg['LR_HEADS']}]
        if backbone_params:
            param_groups.append({'params': backbone_params, 'lr': self.cfg['LR_BACKBONE']})
        return torch.optim.AdamW(param_groups, weight_decay=self.cfg['WEIGHT_DECAY'])

    def train_epoch(self, epoch, loader, desc_suffix=""):
        self.model.train()
        total_loss = 0.0
        valid_batches = 0

        pbar = tqdm(loader, desc=f"Epoch {epoch+1} {desc_suffix}")
        self.optimizer.zero_grad()

        for batch_idx, batch in enumerate(pbar):
            batch_device = {k: v.to(self.device) for k, v in batch.items()
                           if k not in ['motivational_flag', 'extreme_sentiment_flag']}

            if self.cfg['USE_AMP']:
                with autocast():
                    outputs = self.model(batch_device['input_ids'], batch_device['attention_mask'], batch_device['image'])
                    loss, _, _, _ = combined_loss(outputs, batch_device, self.cfg, self.emotion_loss_fn)
                    loss = loss / self.cfg['GRADIENT_ACCUMULATION_STEPS']

                if torch.isnan(loss) or torch.isinf(loss):
                    continue

                self.scaler.scale(loss).backward()

                if (batch_idx + 1) % self.cfg['GRADIENT_ACCUMULATION_STEPS'] == 0:
                    self.scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.cfg['GRADIENT_CLIP'])
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                    self.optimizer.zero_grad()
            else:
                outputs = self.model(batch_device['input_ids'], batch_device['attention_mask'], batch_device['image'])
                loss, _, _, _ = combined_loss(outputs, batch_device, self.cfg, self.emotion_loss_fn)
                loss = loss / self.cfg['GRADIENT_ACCUMULATION_STEPS']

                if torch.isnan(loss) or torch.isinf(loss):
                    continue

                loss.backward()

                if (batch_idx + 1) % self.cfg['GRADIENT_ACCUMULATION_STEPS'] == 0:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.cfg['GRADIENT_CLIP'])
                    self.optimizer.step()
                    self.optimizer.zero_grad()

            loss_val = loss.item() * self.cfg['GRADIENT_ACCUMULATION_STEPS']
            total_loss += loss_val
            valid_batches += 1
            pbar.set_postfix({'loss': f"{loss_val:.4f}"})

        return total_loss / max(valid_batches, 1)

    def validate(self, epoch):
        self.model.eval()
        total_loss = 0.0
        all_sentiment_labels = []
        all_sentiment_outputs = []
        all_emotion_labels = []
        all_emotion_logits = []

        with torch.no_grad():
            for batch in tqdm(self.val_loader, desc=f"Epoch {epoch+1} [Val]"):
                batch_device = {k: v.to(self.device) for k, v in batch.items()
                               if k not in ['motivational_flag', 'extreme_sentiment_flag']}

                outputs = self.model(batch_device['input_ids'], batch_device['attention_mask'], batch_device['image'])
                loss, _, _, _ = combined_loss(outputs, batch_device, self.cfg, self.emotion_loss_fn)
                total_loss += loss.item()

                all_sentiment_labels.append(batch_device['sentiment_label'].cpu())
                all_sentiment_outputs.append({
                    'cumulative_logits': outputs['sentiment']['cumulative_logits'].cpu(),
                    'class_probs': outputs['sentiment']['class_probs'].cpu()
                })
                all_emotion_labels.append(batch_device['emotion_labels'].cpu())
                all_emotion_logits.append(outputs['emotion_logits'].cpu())

        all_sentiment_labels = torch.cat(all_sentiment_labels)
        combined_sentiment = {
            'cumulative_logits': torch.cat([o['cumulative_logits'] for o in all_sentiment_outputs]),
            'class_probs': torch.cat([o['class_probs'] for o in all_sentiment_outputs])
        }
        all_emotion_labels = torch.cat(all_emotion_labels)
        all_emotion_logits = torch.cat(all_emotion_logits)

        if self.cfg.get('TUNE_EMOTION_THRESHOLDS', False):
            print("\n🔧 Tuning emotion thresholds...")
            self.optimal_thresholds = tune_emotion_thresholds(all_emotion_logits, all_emotion_labels)

        metrics = compute_metrics(combined_sentiment, all_sentiment_labels, all_emotion_logits,
                                all_emotion_labels, self.optimal_thresholds)
        avg_loss = total_loss / len(self.val_loader)

        # UPDATED PRINT FORMAT
        print(f"\nValidation Results (Epoch {epoch+1}): "
              f"Loss: {avg_loss:.4f} "
              f"Sentiment Accuracy: {metrics['sentiment_accuracy']:.4f} "
              f"Sentiment F1 (5-class): {metrics['sentiment_f1']:.4f} "
              f"Sentiment F1 (Task A Weighted): {metrics['sentiment_taskA_weighted_f1']:.4f} "
              f"Sentiment MAE: {metrics['sentiment_mae']:.4f} "
              f"Emotion F1 (avg): {metrics['emotion_f1']:.4f} "
              f"humor F1: {metrics['per_emotion_f1'][0]:.4f} "
              f"sarcasm F1: {metrics['per_emotion_f1'][1]:.4f} "
              f"offensive F1: {metrics['per_emotion_f1'][2]:.4f} "
              f"motivational F1: {metrics['per_emotion_f1'][3]:.4f} "
              f"Thresholds: {[f'{t:.3f}' for t in self.optimal_thresholds]}")

        return {**metrics, 'val_loss': avg_loss}
    
    def print_best_results(self):
        """Prints the stored best results at the end of training"""
        if self.best_epoch_metrics is None:
            print("\n⚠️ No best model was saved.")
            return
            
        m = self.best_epoch_metrics
        print(f"\n{'='*80}")
        print(f"🏆 BEST EPOCH RESULTS (Epoch {self.best_epoch_num})")
        print(f"{'='*80}")
        print(f"Validation Results (Epoch {self.best_epoch_num}): "
              f"Loss: {m['val_loss']:.4f} "
              f"Sentiment Accuracy: {m['sentiment_accuracy']:.4f} "
              f"Sentiment F1 (5-class): {m['sentiment_f1']:.4f} "
              f"Sentiment F1 (Task A Weighted): {m.get('sentiment_taskA_weighted_f1', 0.0):.4f} "
              f"Sentiment MAE: {m['sentiment_mae']:.4f} "
              f"Emotion F1 (avg): {m['emotion_f1']:.4f} "
              f"humor F1: {m['per_emotion_f1'][0]:.4f} "
              f"sarcasm F1: {m['per_emotion_f1'][1]:.4f} "
              f"offensive F1: {m['per_emotion_f1'][2]:.4f} "
              f"motivational F1: {m['per_emotion_f1'][3]:.4f} "
              f"Thresholds: {[f'{t:.3f}' for t in self.optimal_thresholds]}")
        print(f"{'='*80}\n")

    def fine_tune_motivational(self):
        print("\n" + "="*70)
        print("STAGE 2: MOTIVATIONAL-ONLY FINE-TUNING")
        print("="*70)
        
        # 1. Load best checkpoint
        checkpoint_path = os.path.join(self.cfg['CHECKPOINT_PATH'], 'best_model_enhanced.pt')
        # FIX: weights_only=False
        checkpoint = torch.load(checkpoint_path, map_location=self.device, weights_only=False)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        
        # 2. Freeze everything except emotion head
        for param in self.model.parameters():
            param.requires_grad = False
        for param in self.model.emotion_head.parameters():
            param.requires_grad = True
            
        # 3. Create Balanced Batch Sampler
        print("Creating strict Balanced Batch Sampler...")
        balanced_sampler = BalancedBatchSampler(
            self.train_loader.dataset, 
            self.cfg['BATCH_SIZE'], 
            pos_ratio=self.cfg['MOTIVATIONAL_BATCH_RATIO']
        )
        balanced_loader = DataLoader(
            self.train_loader.dataset,
            batch_sampler=balanced_sampler, # Use batch_sampler instead of sampler
            num_workers=2,
            pin_memory=True
        )
        
        # 4. Train loop
        self.optimizer = torch.optim.AdamW(
            filter(lambda p: p.requires_grad, self.model.parameters()), 
            lr=1e-4, weight_decay=1e-3
        )
        
        best_ft_metric = -float('inf')
        
        for epoch in range(self.cfg['FINE_TUNE_EPOCHS']):
            real_epoch = self.cfg['EPOCHS'] + epoch + 1
            loss = self.train_epoch(real_epoch, balanced_loader, desc_suffix="[Fine-Tune]")
            print(f"Fine-tune Loss: {loss:.4f}")
            
            metrics = self.validate(real_epoch)
            
            # Metric: Motivational F1 + Emotion Avg
            current_metric = metrics['per_emotion_f1'][3] * 1.5 + metrics['emotion_f1']
            
            # Update BEST TRACKER if fine-tuning improved things
            if current_metric > best_ft_metric:
                best_ft_metric = current_metric
                ft_path = os.path.join(self.cfg['CHECKPOINT_PATH'], 'best_model_finetuned.pt')
                
                # Update global best record for final printing
                self.best_epoch_metrics = metrics
                self.best_epoch_num = real_epoch
                self.optimal_thresholds = self.optimal_thresholds # Ensure latest thresholds are saved
                
                torch.save({
                    'model_state_dict': self.model.state_dict(),
                    'config': self.cfg,
                    'optimal_thresholds': self.optimal_thresholds
                }, ft_path)
                print("✓ Saved fine-tuned model")

    def fit(self):
        print(f"\nSTARTING STAGE 1 TRAINING: {self.cfg['EPOCHS']} EPOCHS")
        
        for epoch in range(self.cfg['EPOCHS']):
            if epoch == self.cfg['UNFREEZE_BACKBONE_EPOCH']:
                print(f"🔓 UNFREEZING BACKBONE at epoch {epoch+1}")
                self.model.unfreeze_backbone(layers_to_unfreeze=self.cfg['UNFREEZE_LAYERS'])
                self.optimizer = self.make_optimizer()
                self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=self.cfg['EPOCHS'])

            loss = self.train_epoch(epoch, self.train_loader)
            print(f"Train Loss: {loss:.4f}")
            val_metrics = self.validate(epoch)
            
            if self.scheduler:
                self.scheduler.step()

            composite = val_metrics['sentiment_f1'] + val_metrics['emotion_f1'] * 2.0
            
            if composite > self.best_metric:
                self.best_metric = composite
                self.counter = 0
                
                # Update global best record
                self.best_epoch_metrics = val_metrics
                self.best_epoch_num = epoch + 1
                
                os.makedirs(self.cfg['CHECKPOINT_PATH'], exist_ok=True)
                checkpoint_path = os.path.join(self.cfg['CHECKPOINT_PATH'], 'best_model_enhanced.pt')
                torch.save({
                    'model_state_dict': self.model.state_dict(),
                    'best_metric': self.best_metric,
                    'config': self.cfg,
                    'optimal_thresholds': self.optimal_thresholds
                }, checkpoint_path)
                print(f"✓ Saved best model (New Record)")
            else:
                self.counter += 1
                if self.counter >= self.patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
        
        # Trigger Fine-Tuning
        self.fine_tune_motivational()
        
        # PRINT FINAL SUMMARY
        self.print_best_results()
        
        return self.best_metric

print("✓ Trainer with Fine-Tuning defined")


# ==================== PART 7: DATA LOADING & SAMPLER ====================
print("\n" + "=" * 80)
print("PART 7: DATA LOADING & BALANCED SAMPLER")
print("=" * 80)

class BalancedBatchSampler(Sampler):
    """
    Custom Sampler that yields batch indices ensuring a fixed ratio of 
    positive motivational samples per batch.
    """
    def __init__(self, dataset, batch_size, pos_ratio=0.30):
        self.dataset = dataset
        self.batch_size = batch_size
        self.n_pos = int(batch_size * pos_ratio)
        self.n_neg = batch_size - self.n_pos
        
        # Separate indices
        self.pos_indices = [i for i, x in enumerate(dataset) if int(x['motivational_flag'].item()) == 1]
        self.neg_indices = [i for i, x in enumerate(dataset) if int(x['motivational_flag'].item()) == 0]
        
        self.n_batches = len(self.neg_indices) // self.n_neg # Based on majority class
        
    def __iter__(self):
        for _ in range(self.n_batches):
            batch = []
            # Sample Positives (with replacement if needed to fill ratios)
            batch.extend(np.random.choice(self.pos_indices, self.n_pos, replace=True))
            # Sample Negatives
            batch.extend(np.random.choice(self.neg_indices, self.n_neg, replace=False))
            np.random.shuffle(batch)
            yield batch

    def __len__(self):
        return self.n_batches

# Initialize tokenizer and transforms
tokenizer = AutoTokenizer.from_pretrained(cfg['TEXT_MODEL'])
train_transform = transforms.Compose([
    transforms.Resize((cfg['IMG_SIZE'], cfg['IMG_SIZE'])),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
])
val_transform = transforms.Compose([
    transforms.Resize((cfg['IMG_SIZE'], cfg['IMG_SIZE'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
])

# Create datasets
train_dataset = MemeDataset(train_df, tokenizer, train_transform, NEW_TRAIN_DIR, cfg)
val_dataset = MemeDataset(val_df, tokenizer, val_transform, NEW_VAL_DIR, cfg)

# Standard weighted sampler for Stage 1
sample_weights = []
for idx in range(len(train_dataset)):
    item = train_dataset[idx]
    weight = cfg['MOTIVATIONAL_OVERSAMPLE_FACTOR'] if item['motivational_flag'].item() == 1 else 1.0
    sample_weights.append(weight)
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=cfg['BATCH_SIZE'], sampler=sampler, num_workers=2, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=cfg['BATCH_SIZE'], shuffle=False, num_workers=2, pin_memory=True)

print(f"✓ Train batches: {len(train_loader)}")
print("✓ Balanced Batch Sampler defined (will be used in Stage 2)")


# ==================== PART 8: MODEL INITIALIZATION & TRAINING ====================
print("\n" + "=" * 80)
print("PART 8: MODEL INITIALIZATION")
print("=" * 80)

model = EnhancedFusionModel(cfg).to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nModel Statistics:")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")

# Initialize enhanced emotion loss
emotion_loss_fn = EnhancedAsymmetricLoss(
    gamma_neg=cfg['ASL_GAMMA_NEG'],
    gamma_pos=cfg['ASL_GAMMA_POS'],
    clip=cfg['ASL_CLIP'],
    priors=cfg['EMO_PRIORS'],
    prior_tau=cfg['ASL_PRIOR_TAU']
)

# Initialize trainer
trainer = Trainer(model, cfg, train_loader, val_loader, device, emotion_loss_fn)

print("\n✓ Trainer initialized")

# ==================== START TRAINING ====================
print("\n" + "=" * 80)
print("STARTING TRAINING")
print("=" * 80)

best_metric = trainer.fit()

print(f"\n{'='*80}")
print(f"✅ TRAINING COMPLETED!")
print(f"{'='*80}")
print(f"Best composite metric: {best_metric:.4f}")
print(f"Fine-tuned Model saved to: {cfg['CHECKPOINT_PATH']}/best_model_finetuned.pt")


# ==================== PART 9: INFERENCE ====================
print("=" * 80)
print("INFERENCE PIPELINE")
print("=" * 80)

class MemeInference:
    def __init__(self, checkpoint_path, device='cuda', thresholds=None):
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        
        print(f"Loading checkpoint from {checkpoint_path}...")
        # FIX: Added weights_only=False
        checkpoint = torch.load(checkpoint_path, map_location=self.device, weights_only=False)
        self.cfg = checkpoint['config']
        
        if thresholds:
            self.current_thresholds = thresholds
        else:
            self.current_thresholds = checkpoint.get('optimal_thresholds', self.cfg['EMO_THRESHOLDS'])
            
        self.model = EnhancedFusionModel(self.cfg).to(self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()
        
        self.tokenizer = AutoTokenizer.from_pretrained(self.cfg['TEXT_MODEL'])
        self.image_transform = transforms.Compose([
            transforms.Resize((self.cfg['IMG_SIZE'], self.cfg['IMG_SIZE'])),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
        ])
        print(f"✓ Thresholds: {[f'{t:.3f}' for t in self.current_thresholds]}")
    
    def preprocess_image(self, image_path):
        try:
            image = Image.open(image_path).convert('RGB')
            return self.image_transform(image)
        except Exception:
            return torch.zeros(3, self.cfg['IMG_SIZE'], self.cfg['IMG_SIZE'])
    
    def preprocess_text(self, text):
        if not text or text.strip() == '': text = 'No text'
        encoding = self.tokenizer(text, max_length=self.cfg['MAX_LEN'], padding='max_length', truncation=True, return_tensors='pt')
        return encoding['input_ids'], encoding['attention_mask']

    @torch.no_grad()
    def predict_single(self, image_path, text):
        image_tensor = self.preprocess_image(image_path).unsqueeze(0).to(self.device)
        input_ids, attention_mask = self.preprocess_text(text)
        input_ids = input_ids.to(self.device)
        attention_mask = attention_mask.to(self.device)
        
        outputs = self.model(input_ids, attention_mask, image_tensor)
        probs = torch.sigmoid(outputs['emotion_logits'])[0]
        thresholds_tensor = torch.tensor(self.current_thresholds, device=self.device)
        preds = (probs >= thresholds_tensor).cpu().numpy()
        probs = probs.cpu().numpy()
        
        sent_probs = outputs['sentiment']['class_probs'][0]
        sent_idx = torch.argmax(sent_probs).item()
        
        emotions = {}
        for k, label in enumerate(self.cfg['EMOTION_LABELS']):
            emotions[label] = {
                'present': bool(preds[k]),
                'probability': float(probs[k]),
                'threshold': float(self.current_thresholds[k])
            }
            
        return {
            'sentiment': self.cfg['SENTIMENT_MAP_REV'][sent_idx],
            'sentiment_conf': float(sent_probs[sent_idx]),
            'emotions': emotions,
            'intensity': float(outputs['intensity'].item())
        }


INSTALLING DEPENDENCIES...


2025-12-02 19:30:47.883431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764703847.904424     221 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764703847.910764     221 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...


🔧 Tuning emotion thresholds...
  Emotion 0: threshold=0.189, Score=0.9224
  Emotion 1: threshold=0.100, Score=0.8818
  Emotion 2: threshold=0.237, Score=0.5767
  Emotion 3: threshold=0.200, Score=0.0394

Validation Results (Epoch 25): Loss: 1.5678 Sentiment Accuracy: 0.3295 Sentiment F1 (5-class): 0.1909 Sentiment F1 (Task A Weighted): 0.4232 Sentiment MAE: 0.9543 Emotion F1 (avg): 0.7706 humor F1: 0.9224 sarcasm F1: 0.8818 offensive F1: 0.5767 motivational F1: 0.0588 Thresholds: ['0.189', '0.100', '0.237', '0.200']


Epoch 26 [Fine-Tune]:   0%|          | 0/437 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Epoch 26 [Fine-Tune]: 100%|██████████| 437/437 [00:39<00:00, 10.97it/s, loss=0.7597]


Fine-tune Loss: 0.7154


Epoch 26 [Val]:   0%|          | 0/66 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Epoch 26 [Val]: 100%|██████████| 66/66 [00:05<00:00, 11.27it/s]



🔧 Tuning emotion thresholds...
  Emotion 0: threshold=0.500, Score=0.0000
  Emotion 1: threshold=0.500, Score=0.0000
  Emotion 2: threshold=0.500, Score=0.0000
  Emotion 3: threshold=0.500, Score=0.0000

Validation Results (Epoch 26): Loss: nan Sentiment Accuracy: 0.3295 Sentiment F1 (5-class): 0.1909 Sentiment F1 (Task A Weighted): 0.4232 Sentiment MAE: 0.9543 Emotion F1 (avg): 0.0000 humor F1: 0.0000 sarcasm F1: 0.0000 offensive F1: 0.0000 motivational F1: 0.0000 Thresholds: ['0.500', '0.500', '0.500', '0.500']

🏆 BEST EPOCH RESULTS (Epoch 22)
Validation Results (Epoch 22): Loss: 1.5482 Sentiment Accuracy: 0.3295 Sentiment F1 (5-class): 0.1909 Sentiment F1 (Task A Weighted): 0.4232 Sentiment MAE: 0.9543 Emotion F1 (avg): 0.7716 humor F1: 0.9224 sarcasm F1: 0.8817 offensive F1: 0.5768 motivational F1: 0.0839 Thresholds: ['0.500', '0.500', '0.500', '0.500']


✅ TRAINING COMPLETED!
Best composite metric: 1.7547
Fine-tuned Model saved to: /kaggle/working/checkpoints/best_model_finetuned

In [ ]:
# ==================== PART 11: INTERACTIVE DEMO (FINAL WORKING VERSION) ====================
import sys
import subprocess

# 1. Install Gradio quietly
print("Installing Gradio...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "gradio"])

import gradio as gr
import numpy as np
from PIL import Image
import os
import torch

print("=" * 80)
print("STARTING INTERACTIVE DEMO")
print("=" * 80)

# 2. Initialize Inference Engine (if not already loaded)
if 'inferencer' not in globals():
    print("Initializing Inference Engine...")
    # Check for the fine-tuned model first, fallback to the base enhanced model
    checkpoint = '/kaggle/working/checkpoints/best_model_finetuned.pt'
    if not os.path.exists(checkpoint):
        print("Fine-tuned model not found, looking for base model...")
        checkpoint = '/kaggle/working/checkpoints/best_model_enhanced.pt'
    
    if os.path.exists(checkpoint):
        inferencer = MemeInference(checkpoint_path=checkpoint, device='cuda')
        print(f"✓ Loaded model from: {checkpoint}")
    else:
        raise FileNotFoundError("No checkpoint found! Did you run the training (Part 8)?")

# 3. Define the Analysis Function
def analyze_meme(image_obj, text_input):
    """
    Takes an image and text, runs the model, and formats the output for Gradio.
    """
    if image_obj is None:
        return "⚠️ Please upload an image.", 0.0, {}
    
    # Save temporary file because predict_single expects a file path
    temp_path = "temp_meme_upload.jpg"
    image_obj.save(temp_path)
    
    try:
        # --- RUN INFERENCE ---
        result = inferencer.predict_single(temp_path, text_input)
        
        # --- FORMAT RESULTS ---
        
        # 1. Sentiment (Fixing the key access error)
        # The model returns 'sentiment' as a simple string, and 'sentiment_conf' as a float
        sent_label = result['sentiment'] 
        sent_conf = result['sentiment_conf']
        sent_str = f"{sent_label.upper()} \n(Confidence: {sent_conf:.1%})"
        
        # 2. Emotions
        emo_out = {}
        for emo, details in result['emotions'].items():
            prob = details['probability']
            is_present = details['present']
            
            # Add a checkmark to the label if the model thinks it's present
            label_str = emo.upper()
            if is_present:
                label_str = f"✅ {label_str}"
            
            emo_out[label_str] = prob
            
        return sent_str, result['intensity'], emo_out
        
    except Exception as e:
        return f"Error during analysis: {str(e)}", 0.0, {}

# 4. Prepare Examples (Robust Loading)
# We try to load 2 images from the validation set to use as clickable examples.
examples_list = []
try:
    if 'val_df' in globals() and 'NEW_VAL_DIR' in globals():
        # Helper to find a valid image path from the dataframe
        def get_valid_path(row, base_dir):
            # Handle different column names for the image filename
            col_name = 'image_name' if 'image_name' in row else row.index[0]
            fname = str(row[col_name])
            if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                fname += ".jpg"
            full_path = os.path.join(base_dir, fname)
            return full_path if os.path.exists(full_path) else None

        # Try to grab the 1st and 6th image from validation
        indices_to_try = [0, 5]
        for idx in indices_to_try:
            if idx < len(val_df):
                path = get_valid_path(val_df.iloc[idx], NEW_VAL_DIR)
                text = str(val_df.iloc[idx].get('text', ''))
                if path:
                    examples_list.append([path, text])
        
        if examples_list:
            print(f"✓ Loaded {len(examples_list)} examples for the demo.")
except Exception as e:
    print(f"⚠️ Could not load auto-examples (minor issue): {e}")

# 5. Build and Launch the Interface
iface = gr.Interface(
    fn=analyze_meme,
    inputs=[
        gr.Image(type="pil", label="1. Upload Meme"), 
        gr.Textbox(label="2. Enter Text (Optional)", placeholder="Type the text inside the meme here...", lines=2)
    ],
    outputs=[
        gr.Label(label="Predicted Sentiment"),
        gr.Number(label="Intensity Score (0.0 - 1.0)"),
        gr.Label(label="Detected Emotions & Probabilities", num_top_classes=4)
    ],
    title="🤖 Enhanced Meme Analysis AI",
    description=f"""
    **Task:** Multimodal Classification (Text + Image Analysis)
    <br><b>Model:</b> Dual-Encoder (MurilBERT + CLIP) with Cross-Attention Fusion.
    <br><b>Current Status:</b> Fine-tuned for Sarcasm & Motivation.
    <br><b>Active Thresholds:</b> {[f'{t:.3f}' for t in inferencer.current_thresholds]}
    """,
    examples=examples_list if examples_list else None,
    theme="default"
)

print("\n🚀 Launching Demo... Click the public link below!")
iface.launch(share=True, debug=True, inline=True)

Installing Gradio...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.8 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


STARTING INTERACTIVE DEMO
Initializing Inference Engine...
Loading checkpoint from /kaggle/working/checkpoints/best_model_finetuned.pt...
✓ Thresholds: ['0.165', '0.197', '0.254', '0.159']
✓ Loaded model from: /kaggle/working/checkpoints/best_model_finetuned.pt
✓ Loaded 2 examples for the demo.

🚀 Launching Demo... Click the public link below!
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://efe476f5aedd5d2e5c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
